# Raman Spectroscopy Decomposition

## Introduction

Once components in a mixture Raman spectra have been identified and assigned, and psudo-Voigt curve fiting has been completed the next step is to compare pure component calibration (or non-decomposing, non-reacting) area under peaks to experimental data. From this comparision one will be able to deterimine:
1. is decomposition occuring?

and if it is then: 
2. calculate the amount of molar decomposition 

This calculation can be completed by comparing the area value of the experimental mixture Raman spectra to the pure component calibration Raman spectra area.

## Pre-step (1/2): Import Modules

In [3]:
#initial imports
import os
import h5py
import pandas as pd
import matplotlib.pyplot as plt
import math
import numpy as np
from ramandecompy import spectrafit
from ramandecompy import peakidentify
from ramandecompy import dataprep
from ramandecompy import dataimport
from ramandecompy import datavis
from ramandecompy import machine_learning

ModuleNotFoundError: No module named 'ramandecompy'

## Pre-step (2/2): Import Calibration / Pure Component Raman Spectra Data Sets

In [ ]:
dataprep.new_hdf5('calibration_data2')

dataprep.add_calibration('calibration_data2.hdf5',
                          '../ramandecompy/tests/test_files/Hydrogen_Baseline_Calibration.xlsx',
                          label='Hydrogen')
dataprep.add_calibration('calibration_data2.hdf5',
                         '../ramandecompy/tests/test_files/CarbonMonoxide_Baseline_Calibration.xlsx',
                         label='CarbonMonoxide')
dataprep.add_calibration('calibration_data2.hdf5','../ramandecompy/tests/test_files/CO2_100wt%.csv',label='CO2')
dataprep.add_calibration('calibration_data2.hdf5','../ramandecompy/tests/test_files/water.xlsx',label='H2O')
dataprep.add_calibration('calibration_data2.hdf5','../ramandecompy/tests/test_files/sapphire.xlsx',label='sapphire')
dataprep.add_calibration('calibration_data2.hdf5','../ramandecompy/tests/test_files/FormicAcid_3_6percent.xlsx',label='FormicAcid')


In [ ]:
dataprep.view_hdf5('calibration_data2.hdf5')

In [ ]:
dataprep.view_hdf5('calibration_data2-Copy1.hdf5')

In [ ]:
dataprep.view_hdf5('dataimport_ML_df-Copy1.hdf5')

## Step 1: Import Experimental Data Sets
The first thing is to put experimental data into a hdf5 file (this file will end up being used to identify peaks)

With multiple files in a directory/ many data sets it is usefull to loop over all files in the directory to add versus adding one by one. The code to loop came from a stackoverflow comment: `https://stackoverflow.com/questions/10377998/how-can-i-iterate-over-files-in-a-given-directory`

Note: A good resource for HDF5 file types in general is: `http://docs.h5py.org/en/stable/`


In [ ]:
#dataprep.new_hdf5('dataprep_experiment') #comment this line out once made for the first time so an error isn't given saying that the file already exists
#directory = '/Users/elizabeth/Desktop/raman-spectra-decomp-analysis/ramandecompy/tests/test_files/' #defining directory for data
#dataprep.view_hdf5('dataprep_experimental.hdf5')

#base_dir = '../ramandecompy/tests/test_files/'

#for filename in os.listdir(directory):
#    if filename.startswith('FA_') and filename.endswith('.csv'):
#        locationandfile = directory + filename
#        dataprep.add_experiment('dataprep_experimental.hdf5', locationandfile)
#        continue
#    else:
#        continue
#return

#FOR CALIBRATION DATA MASS ADD

#dataprep.new_hdf5('dataprep_experiment') #comment this line out once made for the first time so an error isn't given saying that the file already exists
#directory = '/Users/elizabeth/Desktop/raman-spectra-decomp-analysis/ramandecompy/tests/test_files/' #defining directory for data
#


In [ ]:
#type(filename) #checking the type (making sure is a string) for file name

In [ ]:
#dataprep.view_hdf5('dataimport_ML_df-Copy1.hdf5') #making sure the loop did its job and all data is correctly imported 
#comment out this to not see the long list

## Step 2: Define substance of interest
The second step is to determine if the desired speciecies in the spectra is present, and if it is then if it has decomposed (decreased/changed) from the defined calibration area. 

At this point this will be done by the user knowing where the approximate location of the peak for the substance that is of interest. 

Given the user center peak wavelength location input the code will go through the calibration data and for a peak with a center at the defined location (ith some tolerance of +/- 10 cm^-1) will take the area of that curve and store it as a variable.

In [ ]:
#getting peak information for Formic Acid 

data1 = h5py.File('calibration_data2.hdf5', 'r+')
# then specify the peak
peak_01 = list(data1['FormicAcid/Peak_01'][0])
#peak_01s = data1['FormicAcid/Peak_01']
# you put list because otherwise it just saves it as a h5py.dataset or something and lists are more familiar. Then peak_01 will be a list containing the 7 elements of the Peak_01 dataset
print(peak_01)

type(peak_01)
#print(type(peak_01s))

In [ ]:
peak_01[6] #Looking for area under the curve value for the first peak

### Figuring out what peak from Formic Acid to use/ are close to other observed peaks used in analysis
For Formic Acid, prior reports of the wavenumbers of significant Raman Peaks (cm^-1) were at:
- 712
- 1219
- 1400
- 1714
- 2943

In [ ]:
peak_01 = list(data1['FormicAcid/Peak_01'][0])
print(peak_01[2])

In [ ]:
peak_02 = list(data1['FormicAcid/Peak_02'][0])
print(peak_02[2])

In [ ]:
peak_03 = list(data1['FormicAcid/Peak_03'][0])
print(peak_03[2])

In [ ]:
peak_04 = list(data1['FormicAcid/Peak_04'][0])
print(peak_04[2])

In [ ]:
peak_05 = list(data1['FormicAcid/Peak_05'][0])
print(peak_05[2])

In [ ]:
peak_06 = list(data1['FormicAcid/Peak_06'][0])
print(peak_06[2])

### All peaks previously reported are identified in the calibration file within +/- 5 wavenumbers
There is an additional peak identified at 1055, but it is hypothesized that this peak may not be easily identified from other components with similar wavenumbers and/or with the amplitude of the peak being smaller then peaks of formic acid at the other wavenumbers this could be why it isn't identifed in literature. 

For this example the peak occuring at 1400 cm^-1 (peak_04) will be used for molar concentration calculations **because... NEED TO FILL IN**

In [ ]:
FA_cal_area = peak_04[6]
print(FA_cal_area)

## Step 3: Define presence of substance in experimental data

Then for that same center peak wavelength location input it will identify the presence of the peak (if it is there) in the experimental data and area for that peak and store it as a second variable.


In [ ]:
#define filenames
hdf5_calfilename = 'calibration_data2.hdf5' #update to hdf5_calfilename
hdf5_expfilename = 'dataimport_ML_df-Copy1.hdf5'

cal_key_list = machine_learning.keyfinder(hdf5_calfilename)
exp_key_list = machine_learning.keyfinder(hdf5_expfilename)

In [ ]:
print(cal_key_list)

In [ ]:
print(exp_key_list)

In [ ]:
key_list= machine_learning.keyfinder('dataimport_ML_df-Copy1.hdf5')
key_list

In [ ]:
hdf5_calfilename = 'calibration_data2.hdf5' #update to hdf5_calfilename
hdf5_expfilename = 'dataimport_ML_df-Copy2.hdf5'

df = peakidentify.peak_assignment(hdf5_expfilename, '300C/25s', hdf5_calfilename, 10)

In [ ]:
hdf5_calfilename = 'calibration_data2.hdf5' #update to hdf5_calfilename
hdf5_expfilename = 'dataimport_ML_df-Copy1.hdf5'

frames = []
for i, key in enumerate(key_list):
    df = peakidentify.peak_assignment(hdf5_expfilename, '300C/25s', hdf5_calfilename, 10)
    frames.append(df)


In [ ]:
result = pd.concat(frames,axis=0, join='outer', join_axes=None, ignore_index=False,
         keys=None, levels=None, names=None, verify_integrity=False,
         copy=True,sort=True)


dat = pd.DataFrame()


dat['fraction']= result[:][0]
dat['sigma']= result[:][1]
# dat['center']= result[:][2]
dat['amplitude']= result[:][3]
dat['fwhm']= result[:][4]
dat['height']= result[:][5]
dat['auc']= result[:][6]
dat['labelencoded']=le.fit_transform(result[:][7])
dat['labels']= result[:][7]
dat['center']= result[:][2]

In [ ]:
for exp_key_list in 'dataimport_ML_df-Copy1.hdf5':
    for 

## Step 5: Calculate Molar Decomposition

To define the molar decomposition the area of the experimental data will be divided by the calibration data's area. This value will be the molar amount of the substance at the given experimental temperature and resonance time.

## Step 6: Plot Molar Decomposition

## Step 7: Compare Molar Decomposition with Reported Literature Values

# Conclusion